In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Info'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "More Information for")]')))
                list_tr = browser.find_elements(by=By.XPATH, value='//section[@aria-label="Components that come in this kit"]/descendant::tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-1") or contains(@class, "altrow-b-0"))]/tr')

                # = = = = = = = = = = = = = = =
                
                if len(list_tr) == 0:
                    break
                
                # = = = = = = = = = = = = = = =
                
                list_no = [_ + 1 for _ in range(len(list_tr))]
                list_quantity = []
                list_note = []
                list_manufacturer = []
                list_part_number = []
                list_info = []
                for _ in list_tr:
                    list_td = _.find_elements(by=By.XPATH, value='./td')
                    list_quantity.append(list_td[0].text)
                    list_note.append(list_td[1].text)
                    list_manufacturer.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-manufacturer")]').text)
                    list_part_number.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-partnumber")]').text)
                    list_info.append(list_td[2].find_element(by=By.XPATH, value='./descendant::a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]').get_attribute('href'))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Info': url,
                                        'Kit_序号': list_no,
                                        'Kit_Quantity': list_quantity,
                                        'Kit_Note': list_note,
                                        'Kit_Manufacturer': list_manufacturer,
                                        'Kit_Part_Number': list_part_number,
                                        'Kit_OE': '',
                                        'Kit_Pic': '',
                                        'Kit_Info': list_info,
                                        'Kit_Src': ''})
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./kit_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：445

https://www.rockauto.com/en/moreinfo.php?pk=8304972&cc=0&pt=5756  <->  [error] - 剩余数量：430
https://www.rockauto.com/en/moreinfo.php?pk=9722076&cc=0&pt=5756  <->  [ok] - 剩余数量：429
https://www.rockauto.com/en/moreinfo.php?pk=8304992&cc=0&pt=5756  <->  [ok] - 剩余数量：428
https://www.rockauto.com/en/moreinfo.php?pk=10110000&cc=0&pt=5756  <->  [ok] - 剩余数量：427
https://www.rockauto.com/en/moreinfo.php?pk=8304976&cc=0&pt=5756  <->  [ok] - 剩余数量：426
https://www.rockauto.com/en/moreinfo.php?pk=10454500&cc=0&pt=5756  <->  [ok] - 剩余数量：425
https://www.rockauto.com/en/moreinfo.php?pk=8304984&cc=0&pt=5756  <->  [ok] - 剩余数量：424
https://www.rockauto.com/en/moreinfo.php?pk=13570025&cc=0&pt=5756  <->  [ok] - 剩余数量：423
https://www.rockauto.com/en/moreinfo.php?pk=8305000&cc=0&pt=5756  <->  [ok] - 剩余数量：422
https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756  <->  [ok] - 剩余数量：421
https://www.rockauto.com/en/moreinfo.php?pk=8304980&cc=0&pt=5756  <->  [ok] - 剩余数量：420
https://www.rockauto.com/en/

https://www.rockauto.com/en/moreinfo.php?pk=8305064&cc=0&pt=5756  <->  [ok] - 剩余数量：336
https://www.rockauto.com/en/moreinfo.php?pk=5629017&cc=0&pt=5756  <->  [ok] - 剩余数量：335
https://www.rockauto.com/en/moreinfo.php?pk=8305060&cc=0&pt=5756  <->  [ok] - 剩余数量：334
https://www.rockauto.com/en/moreinfo.php?pk=8305080&cc=0&pt=5756  <->  [ok] - 剩余数量：333
https://www.rockauto.com/en/moreinfo.php?pk=8305084&cc=0&pt=5756  <->  [ok] - 剩余数量：332
https://www.rockauto.com/en/moreinfo.php?pk=7075180&cc=0&pt=5756  <->  [ok] - 剩余数量：331
https://www.rockauto.com/en/moreinfo.php?pk=8305100&cc=0&pt=5756  <->  [ok] - 剩余数量：330
https://www.rockauto.com/en/moreinfo.php?pk=8305092&cc=0&pt=5756  <->  [ok] - 剩余数量：329
https://www.rockauto.com/en/moreinfo.php?pk=8305104&cc=0&pt=5756  <->  [ok] - 剩余数量：328
https://www.rockauto.com/en/moreinfo.php?pk=8305108&cc=0&pt=5756  <->  [ok] - 剩余数量：327
https://www.rockauto.com/en/moreinfo.php?pk=8305076&cc=0&pt=5756  <->  [ok] - 剩余数量：326
https://www.rockauto.com/en/moreinfo.php?pk

https://www.rockauto.com/en/moreinfo.php?pk=11492493&cc=0&pt=5756  <->  [error] - 剩余数量：242
https://www.rockauto.com/en/moreinfo.php?pk=8305368&cc=0&pt=5756  <->  [ok] - 剩余数量：241
https://www.rockauto.com/en/moreinfo.php?pk=8305372&cc=0&pt=5756  <->  [ok] - 剩余数量：240
https://www.rockauto.com/en/moreinfo.php?pk=8305364&cc=0&pt=5756  <->  [ok] - 剩余数量：239
https://www.rockauto.com/en/moreinfo.php?pk=11158137&cc=0&pt=5756  <->  [error] - 剩余数量：238
https://www.rockauto.com/en/moreinfo.php?pk=8305376&cc=0&pt=5756  <->  [ok] - 剩余数量：237
https://www.rockauto.com/en/moreinfo.php?pk=11158145&cc=0&pt=5756  <->  [error] - 剩余数量：236
https://www.rockauto.com/en/moreinfo.php?pk=11158141&cc=0&pt=5756  <->  [error] - 剩余数量：235
https://www.rockauto.com/en/moreinfo.php?pk=8305380&cc=0&pt=5756  <->  [ok] - 剩余数量：234
https://www.rockauto.com/en/moreinfo.php?pk=9658992&cc=0&pt=5756  <->  [ok] - 剩余数量：233
https://www.rockauto.com/en/moreinfo.php?pk=11492497&cc=0&pt=5756  <->  [error] - 剩余数量：232
https://www.rockauto.co

https://www.rockauto.com/en/moreinfo.php?pk=7302592&cc=0&pt=5756  <->  [ok] - 剩余数量：148
https://www.rockauto.com/en/moreinfo.php?pk=5031162&cc=0&pt=5756  <->  [ok] - 剩余数量：147
https://www.rockauto.com/en/moreinfo.php?pk=8305608&cc=0&pt=5756  <->  [ok] - 剩余数量：146
https://www.rockauto.com/en/moreinfo.php?pk=8305612&cc=0&pt=5756  <->  [ok] - 剩余数量：145
https://www.rockauto.com/en/moreinfo.php?pk=8305460&cc=0&pt=5756  <->  [ok] - 剩余数量：144
https://www.rockauto.com/en/moreinfo.php?pk=8305620&cc=0&pt=5756  <->  [ok] - 剩余数量：143
https://www.rockauto.com/en/moreinfo.php?pk=8305628&cc=0&pt=5756  <->  [ok] - 剩余数量：142
https://www.rockauto.com/en/moreinfo.php?pk=8305632&cc=0&pt=5756  <->  [ok] - 剩余数量：141
https://www.rockauto.com/en/moreinfo.php?pk=8305616&cc=0&pt=5756  <->  [ok] - 剩余数量：140
https://www.rockauto.com/en/moreinfo.php?pk=8305636&cc=0&pt=5756  <->  [ok] - 剩余数量：139
https://www.rockauto.com/en/moreinfo.php?pk=8305640&cc=0&pt=5756  <->  [ok] - 剩余数量：138
https://www.rockauto.com/en/moreinfo.php?pk

https://www.rockauto.com/en/moreinfo.php?pk=8306052&cc=0&pt=5756  <->  [ok] - 剩余数量：53
https://www.rockauto.com/en/moreinfo.php?pk=8306200&cc=0&pt=5756  <->  [ok] - 剩余数量：52
https://www.rockauto.com/en/moreinfo.php?pk=7096440&cc=0&pt=5756  <->  [ok] - 剩余数量：51
https://www.rockauto.com/en/moreinfo.php?pk=8306156&cc=0&pt=5756  <->  [ok] - 剩余数量：50
https://www.rockauto.com/en/moreinfo.php?pk=8715968&cc=0&pt=5756  <->  [error] - 剩余数量：49
https://www.rockauto.com/en/moreinfo.php?pk=8715972&cc=0&pt=5756  <->  [error] - 剩余数量：48
https://www.rockauto.com/en/moreinfo.php?pk=8715980&cc=0&pt=5756  <->  [error] - 剩余数量：47
https://www.rockauto.com/en/moreinfo.php?pk=8306204&cc=0&pt=5756  <->  [ok] - 剩余数量：46
https://www.rockauto.com/en/moreinfo.php?pk=8715984&cc=0&pt=5756  <->  [error] - 剩余数量：45
https://www.rockauto.com/en/moreinfo.php?pk=9854500&cc=0&pt=5756  <->  [ok] - 剩余数量：44
https://www.rockauto.com/en/moreinfo.php?pk=8715988&cc=0&pt=5756  <->  [error] - 剩余数量：43
https://www.rockauto.com/en/moreinfo.ph

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
